In [1]:
from __future__ import print_function

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import string
printable = set(string.printable)

In [2]:
# load product descriptions
data = pd.read_pickle("./Data/abstracts")

data = data[["abstract"]].values.flatten().tolist()

clean_data = [filter(lambda x: x in printable, s) for s in data]

print(clean_data[0])

docs_raw = [i.decode('utf-8') for i in clean_data]


We have extensively mapped a sample of dense molecular clouds (L1512, TMC-1C,
L1262, Per 7, L1389, L1251E) in lines of HC3N, CH3OH, SO and C^{18}O. We
demonstrate that a high degree of chemical differentiation is present in all of
the observed clouds. We analyse the molecular maps for each cloud,
demonstrating a systematic chemical differentiation across the sample, which we
relate to the evolutionary state of the cloud. We relate our observations to
the cloud physical, kinematical and evolutionary properties, and also compare
them to the predictions of simple chemical models. The implications of this
work for understanding the origin of the clumpy structures and chemical
differentiation observed in dense clouds are discussed.


In [3]:
# Initilise CountVectorizer and fit to the descriptions data
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 1.0,
                                min_df = 100,
                                ngram_range = (2, 3))
dtm_tf = tf_vectorizer.fit_transform(docs_raw)

In [4]:
# Transform the product descpritons using term-frequency-inverse-document-frequency
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(docs_raw)

In [5]:
# Train LDA on the vectorised descriptions
lda_tfidf = LatentDirichletAllocation(n_topics=10, random_state=0, n_jobs=8,  max_iter=20, learning_method="online",batch_size=5000)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(batch_size=5000, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=20, mean_change_tol=0.001,
             n_jobs=8, n_topics=10, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [6]:
# plot the visualisation
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      12.700282        1       1  0.174151 -0.080833
1      11.910327        1       2 -0.320098  0.085055
3      11.763529        1       3  0.111491  0.196521
7      11.068673        1       4  0.047578 -0.223236
6      10.803606        1       5  0.122507 -0.039965
5      10.382094        1       6  0.029479  0.189530
0       8.770668        1       7 -0.037391 -0.185071
8       7.961166        1       8 -0.211888 -0.089236
2       7.319870        1       9 -0.004328  0.102724
9       7.319785        1      10  0.088501  0.044510, topic_info=     Category        Freq                         Term       Total  loglift  \
term                                                                          
80    Default  683.000000                  dark matter  683.000000  30.0000   
283   Default  523.000000               magnetic field  523.000000  29.0000   
167   Default  401.000000                    gamma ray  401.000000  28.0000   
519   Default  465.000000               star formation  465.000000  27.0000   
32    Default  431.000000                   black hole  431.000000  26.0000   
397   Default  239.000000               power spectrum  239.000000  25.0000   
348   Default  203.000000                neutron stars  203.000000  24.0000   
196   Default  250.000000                  high energy  250.000000  23.0000   
347   Default  244.000000                 neutron star  244.000000  22.0000   
35    Default  248.000000                  black holes  248.000000  21.0000   
285   Default  228.000000              magnetic fields  228.000000  20.0000   
79    Default  218.000000                  dark energy  218.000000  19.0000   
327   Default  155.000000         microwave background  155.000000  18.0000   
251   Default  229.000000                 light curves  229.000000  17.0000   
497   Default  160.000000              space telescope  160.000000  16.0000   
67    Default  144.000000             cosmic microwave  144.000000  15.0000   
68    Default  142.000000  cosmic microwave background  142.000000  14.0000   
462   Default  157.000000                 scalar field  157.000000  13.0000   
523   Default  216.000000                 star forming  216.000000  12.0000   
516   Default  154.000000               standard model  154.000000  11.0000   
128   Default  152.000000               equation state  152.000000  10.0000   
140   Default  124.000000                   field view  124.000000   9.0000   
69    Default  147.000000                   cosmic ray  147.000000   8.0000   
70    Default  130.000000                  cosmic rays  130.000000   7.0000   
188   Default  208.000000           gravitational wave  208.000000   6.0000   
250   Default  163.000000                  light curve  163.000000   5.0000   
585   Default  149.000000                  white dwarf  149.000000   4.0000   
433   Default  152.000000                 ray emission  152.000000   3.0000   
77    Default  104.000000                cross section  104.000000   2.0000   
219   Default   94.000000                 hubble space   94.000000   1.0000   
...       ...         ...                          ...         ...      ...   
15    Topic10   64.116567                    agb stars   72.139457   2.4967   
140   Topic10  105.612754                   field view  124.099827   2.4533   
374   Topic10   45.871293               paper presents   54.388241   2.4443   
497   Topic10  128.447371              space telescope  160.687460   2.3906   
424   Topic10   52.338108              radio telescope   67.348589   2.3624   
13    Topic10   58.039059              adaptive optics   80.386844   2.2889   
586   Topic10   77.927080                 white dwarfs  108.446926   2.2841   
207   Topic10   31.853007                 high spatial   44.767795   2.2742   
508   Topic10   44.468418          spectral resolution   63.039991   2

In [7]:
# Train LDA using a higher number of topics
lda_tfidf_large = LatentDirichletAllocation(n_topics=25, random_state=0, n_jobs=8,  max_iter=20, learning_method="online", batch_size=5000)
lda_tfidf_large.fit(dtm_tfidf)

LatentDirichletAllocation(batch_size=5000, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=20, mean_change_tol=0.001,
             n_jobs=8, n_topics=25, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [8]:
# plot the visualisation
pyLDAvis.sklearn.prepare(lda_tfidf_large, dtm_tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
0      5.919929        1       1 -0.111807  0.196164
16     5.903871        1       2 -0.158165  0.040697
5      5.093427        1       3 -0.034045 -0.079250
24     5.032185        1       4 -0.142460 -0.190822
4      5.004474        1       5 -0.168496  0.138955
23     4.985245        1       6 -0.147290  0.145960
6      4.923682        1       7 -0.139501  0.019720
13     4.601002        1       8  0.047249  0.181053
10     4.585166        1       9 -0.017648 -0.277721
8      4.184060        1      10  0.168041 -0.007551
14     3.950442        1      11 -0.178958  0.099723
11     3.903657        1      12 -0.098445 -0.206941
7      3.845099        1      13  0.259340  0.025867
12     3.761222        1      14 -0.033924 -0.272091
20     3.743767        1      15  0.021394 -0.072762
18     3.723759        1      16  0.289195 -0.021042
21     3.678470        1      17 -0.003686 -0.023995
22     3.557791        1      18 -0.048378  0.118985
1      3.417087        1      19  0.276694  0.040768
17     3.158142        1      20  0.182151  0.066598
19     2.926666        1      21 -0.014264  0.076736
9      2.735986        1      22  0.198622  0.025761
15     2.548544        1      23 -0.101869 -0.015829
2      2.528256        1      24 -0.028285  0.029470
3      2.288069        1      25 -0.015464 -0.038452, topic_info=     Category        Freq                         Term       Total  loglift  \
term                                                                          
283   Default  531.000000               magnetic field  531.000000  30.0000   
80    Default  610.000000                  dark matter  610.000000  29.0000   
32    Default  421.000000                   black hole  421.000000  28.0000   
167   Default  417.000000                    gamma ray  417.000000  27.0000   
21    Default  241.000000             angular momentum  241.000000  26.0000   
285   Default  232.000000              magnetic fields  232.000000  25.0000   
519   Default  439.000000               star formation  439.000000  24.0000   
397   Default  270.000000               power spectrum  270.000000  23.0000   
35    Default  240.000000                  black holes  240.000000  22.0000   
327   Default  172.000000         microwave background  172.000000  21.0000   
196   Default  255.000000                  high energy  255.000000  20.0000   
462   Default  164.000000                 scalar field  164.000000  19.0000   
67    Default  160.000000             cosmic microwave  160.000000  18.0000   
68    Default  158.000000  cosmic microwave background  158.000000  17.0000   
585   Default  160.000000                  white dwarf  160.000000  16.0000   
79    Default  234.000000                  dark energy  234.000000  15.0000   
348   Default  185.000000                neutron stars  185.000000  14.0000   
251   Default  219.000000                 light curves  219.000000  13.0000   
347   Default  231.000000                 neutron star  231.000000  12.0000   
188   Default  209.000000           gravitational wave  209.000000  11.0000   
69    Default  151.000000                   cosmic ray  151.000000  10.0000   
70    Default  133.000000                  cosmic rays  133.000000   9.0000   
586   Default  116.000000                 white dwarfs  116.000000   8.0000   
140   Default  129.000000                   field view  129.000000   7.0000   
484   Default  118.000000                   sky survey  118.000000   6.0000   
523   Default  205.000000                 star forming  205.000000   5.0000   
9     Default  117.000000              active galactic  117.000000   4.0000   
516   Default  141.000000               standard model  141.000000   3.0000   
250   Default  157.000000                  light curve  157.000000   2.0000   
235   Default  130.000000                   lambda cdm  130.000000   1.0000  

In [9]:
pyLDAvis.sklearn.prepare(lda_tfidf_large, dtm_tfidf, tfidf_vectorizer, mds='mmds')

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
0      5.919929        1       1  0.022297 -0.244402
16     5.903871        1       2 -0.203839 -0.162037
5      5.093427        1       3 -0.228746  0.234879
24     5.032185        1       4 -0.274765  0.039861
4      5.004474        1       5 -0.107122 -0.239680
23     4.985245        1       6  0.141604 -0.323897
6      4.923682        1       7  0.036719 -0.005993
13     4.601002        1       8  0.200407 -0.077407
10     4.585166        1       9 -0.424638  0.093703
8      4.184060        1      10  0.424577  0.004214
14     3.950442        1      11  0.034787 -0.375554
11     3.903657        1      12 -0.378979 -0.037123
7      3.845099        1      13  0.376579  0.186021
12     3.761222        1      14 -0.137627  0.237879
20     3.743767        1      15  0.304067 -0.272746
18     3.723759        1      16 -0.003250  0.462493
21     3.678470        1      17  0.336842 -0.140457
22     3.557791        1      18 -0.061533 -0.077539
1      3.417087        1      19  0.125191  0.304927
17     3.158142        1      20  0.292969  0.247317
19     2.926666        1      21 -0.161234 -0.419145
9      2.735986        1      22  0.180574  0.404365
15     2.548544        1      23  0.149823  0.118245
2      2.528256        1      24 -0.342952 -0.267771
3      2.288069        1      25 -0.301748  0.309846, topic_info=     Category        Freq                         Term       Total  loglift  \
term                                                                          
283   Default  531.000000               magnetic field  531.000000  30.0000   
80    Default  610.000000                  dark matter  610.000000  29.0000   
32    Default  421.000000                   black hole  421.000000  28.0000   
167   Default  417.000000                    gamma ray  417.000000  27.0000   
21    Default  241.000000             angular momentum  241.000000  26.0000   
285   Default  232.000000              magnetic fields  232.000000  25.0000   
519   Default  439.000000               star formation  439.000000  24.0000   
397   Default  270.000000               power spectrum  270.000000  23.0000   
35    Default  240.000000                  black holes  240.000000  22.0000   
327   Default  172.000000         microwave background  172.000000  21.0000   
196   Default  255.000000                  high energy  255.000000  20.0000   
462   Default  164.000000                 scalar field  164.000000  19.0000   
67    Default  160.000000             cosmic microwave  160.000000  18.0000   
68    Default  158.000000  cosmic microwave background  158.000000  17.0000   
585   Default  160.000000                  white dwarf  160.000000  16.0000   
79    Default  234.000000                  dark energy  234.000000  15.0000   
348   Default  185.000000                neutron stars  185.000000  14.0000   
251   Default  219.000000                 light curves  219.000000  13.0000   
347   Default  231.000000                 neutron star  231.000000  12.0000   
188   Default  209.000000           gravitational wave  209.000000  11.0000   
69    Default  151.000000                   cosmic ray  151.000000  10.0000   
70    Default  133.000000                  cosmic rays  133.000000   9.0000   
586   Default  116.000000                 white dwarfs  116.000000   8.0000   
140   Default  129.000000                   field view  129.000000   7.0000   
484   Default  118.000000                   sky survey  118.000000   6.0000   
523   Default  205.000000                 star forming  205.000000   5.0000   
9     Default  117.000000              active galactic  117.000000   4.0000   
516   Default  141.000000               standard model  141.000000   3.0000   
250   Default  157.000000                  light curve  157.000000   2.0000   
235   Default  130.000000                   lambda cdm  130.000000   1.0000  

In [10]:
# for TF DTM (not inverse document frequency)
lda_tf = LatentDirichletAllocation(n_topics=25, random_state=0, n_jobs=4,max_iter=20, learning_method="online",batch_size=5000)
lda_tf.fit(dtm_tf)

LatentDirichletAllocation(batch_size=5000, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=20, mean_change_tol=0.001,
             n_jobs=4, n_topics=25, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [11]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
14     7.976365        1       1  0.258063 -0.008718
8      6.386764        1       2  0.026794  0.178650
1      6.292388        1       3 -0.092454  0.274945
5      5.587668        1       4 -0.086926  0.045928
10     5.535260        1       5 -0.223143 -0.230874
16     4.699565        1       6  0.123039 -0.095773
22     4.635152        1       7  0.172312 -0.023447
2      4.457123        1       8 -0.121852  0.250875
18     4.227649        1       9 -0.155677 -0.206575
6      4.172334        1      10  0.024555 -0.039689
15     4.016973        1      11 -0.072549  0.045176
20     3.800630        1      12 -0.086232  0.057480
24     3.793421        1      13  0.250299 -0.063939
4      3.761021        1      14  0.105018 -0.081493
23     3.659004        1      15  0.217693 -0.045902
7      3.221226        1      16  0.035366  0.021686
0      3.220892        1      17  0.090350  0.071088
21     3.030239        1      18  0.009277  0.016910
12     2.956527        1      19 -0.039096  0.028310
13     2.902196        1      20 -0.118017 -0.117470
17     2.871648        1      21 -0.097323 -0.140015
19     2.853010        1      22  0.036711  0.046037
11     2.242752        1      23 -0.093227  0.048197
9      1.885596        1      24 -0.102621 -0.012632
3      1.814598        1      25 -0.060358 -0.018754, topic_info=     Category         Freq                        Term        Total  loglift  \
term                                                                           
80    Default  3581.000000                 dark matter  3581.000000  30.0000   
283   Default  2475.000000              magnetic field  2475.000000  29.0000   
32    Default  2479.000000                  black hole  2479.000000  28.0000   
519   Default  3019.000000              star formation  3019.000000  27.0000   
167   Default  2393.000000                   gamma ray  2393.000000  26.0000   
35    Default  1175.000000                 black holes  1175.000000  25.0000   
245   Default  1359.000000                 large scale  1359.000000  24.0000   
21    Default   811.000000            angular momentum   811.000000  23.0000   
397   Default  1060.000000              power spectrum  1060.000000  22.0000   
269   Default  1266.000000                    low mass  1266.000000  21.0000   
251   Default   908.000000                light curves   908.000000  20.0000   
285   Default   889.000000             magnetic fields   889.000000  19.0000   
523   Default  1294.000000                star forming  1294.000000  18.0000   
79    Default   906.000000                 dark energy   906.000000  17.0000   
196   Default  1024.000000                 high energy  1024.000000  16.0000   
347   Default   972.000000                neutron star   972.000000  15.0000   
530   Default  1289.000000                stellar mass  1289.000000  14.0000   
288   Default   784.000000               main sequence   784.000000  13.0000   
188   Default   804.000000          gravitational wave   804.000000  12.0000   
348   Default   664.000000               neutron stars   664.000000  11.0000   
339   Default   523.000000                 monte carlo   523.000000  10.0000   
330   Default  1032.000000                   milky way  1032.000000   9.0000   
497   Default   553.000000             space telescope   553.000000   8.0000   
300   Default   557.000000                   mass loss   557.000000   7.0000   
250   Default   618.000000                 light curve   618.000000   6.0000   
9     Default   567.000000             active galactic   567.000000   5.0000   
585   Default   533.000000                 white dwarf   533.000000   4.0000   
327   Default   607.000000        microwave background   607.000000   3.0000   
433   Default   791.000000                ray emission   791.000000   2.0000   
156   Default   502.000000             galacti

In [12]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [13]:
lda_tfidf_large.fit(dtm_tf)

n_top_words = 4
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda_tfidf_large, tf_feature_names, n_top_words)


Topics in LDA model:
Topic #0:
large scale, early type, main sequence, scale structure
Topic #1:
power spectrum, microwave background, cosmic microwave, cosmic microwave background
Topic #2:
dark energy, lambda cdm, equation state, young stellar
Topic #3:
angular momentum, black hole, accretion rate, accretion disk
Topic #4:
low mass, emission line, emission lines, mass stars
Topic #5:
black hole, black holes, gravitational wave, supermassive black
Topic #6:
light curves, light curve, radial velocity, long term
Topic #7:
milky way, magnetic fields, power law, dwarf galaxies
Topic #8:
dark matter, standard model, cross section, cold dark
Topic #9:
monte carlo, low frequency, cosmological constant, type supernovae
Topic #10:
magnetic field, gamma ray, high energy, cosmic ray
Topic #11:
active galactic, galactic nuclei, active galactic nuclei, high redshift
Topic #12:
non thermal, radio emission, galactic center, time scale
Topic #13:
numerical simulations, luminosity function, spectral 

In [14]:
print type()

SyntaxError: invalid syntax (<ipython-input-14-e6ecb3b8531b>, line 1)